In [2]:
!pip install arxiv

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/81.3 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.3 kB 445.2 kB/s eta 0:00:01
   --------------- ------------------------ 30.7/81.3 kB 445.2 kB/s eta 0:00:01
   ------------------------------ --------- 61.4/81.3 kB 365.7 kB/s eta 0:00:01
   ----------------------------------- ---- 71.7/81.3 kB 328.6 kB/s eta 0:00:01
   ---------------------------------------- 81.3/81.3 kB 326.2 kB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=0387a3b3cfb159b2d8416aa6ba692eeca0502ef


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [5]:
api_wrapper = WikipediaAPIWrapper(top_k_results=2, doc_content_chars_max=200)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
wikipedia_tool.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/langsmith/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024E041E48D0>, search_kwargs={})

In [12]:
## Creating a retrieval tool
from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information on langsmith, for any question on langsmith, you must use this tool")
retrieval_tool.name



'langsmith_search'

In [13]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=2, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name


'arxiv'

In [14]:
tools = [retrieval_tool, arxiv_tool]
tools


[Tool(name='langsmith_search', description='Search for information on langsmith, for any question on langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000024E71D28AE0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024E041E48D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000024E71D291C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024E041E48D0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{pag

In [19]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [22]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
# Now to create an agent
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent


RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [25]:
# Agent executer

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Tell me about langsmith"})




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'langsmith'}`


🦜🛠️ LangSmith | 🦜️🔗 LangChain

Skip to main contentA newer LangChain version is out! Check out the latest version.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpression LanguageGet startedRunnable interfacePrimitivesAdvantages of LCELStreamingAdd message history (memory)MoreEcosystem🦜🛠️ LangSmith🦜🛠️ LangSmithLangSmith Walkthrough🦜🕸️ LangGraph🦜️🏓 LangServeSecurityThis is documentation for LangChain v0.1, which is no longer actively maintained.For the current stable version, see this version (Latest).Ecosystem🦜🛠️ LangSmith🦜🛠️ LangSmithLangSmith helps you 

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a tool that helps trace and evaluate language model applications and intelligent agents to assist in transitioning from prototype to production. It provides support for various functionalities like extracting structured output, chatbots, tool use, agents, query analysis, and more. LangSmith offers interactive walkthroughs, documentation, tutorials, and end-to-end examples to integrate it into workflows effectively. You can find more information on LangSmith in the LangSmith documentation and LangSmith Cookbook.'}

In [26]:
agent_executor.invoke({"input": "Tell me about machine learning"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'machine learning'}`


move from prototype to production.Check out the interactive walkthrough to get started.For more information, please refer to the LangSmith documentation.For tutorials and other end-to-end examples demonstrating ways to integrate LangSmith in your workflow,
check out the LangSmith Cookbook. Some of the guides therein include:Leveraging user feedback in your JS application (link).Building an automated feedback pipeline (link).How to evaluate and audit your RAG workflows (link).How to fine-tune an LLM on real usage data (link).How to use the LangChain Hub to version your prompts (link)Help us out by providing feedback on this documentation page:PreviousMultiple chainsNext🦜🛠️ LangSmithCommunityDiscordTwitterGitHubPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024 LangChain, Inc.

Skip to main contentA newer LangChain version is out! Check out the latest version.ComponentsIntegration

{'input': 'Tell me about machine learning',
 'output': 'Machine learning is a branch of artificial intelligence that focuses on the development of algorithms and models that enable computers to learn and make decisions based on data without being explicitly programmed. It involves the use of statistical techniques to give computers the ability to "learn" from data and improve their performance on a specific task over time. Machine learning is widely used in various fields such as image recognition, natural language processing, and predictive analytics. If you would like more detailed information or have specific questions about machine learning, feel free to ask!'}

In [27]:
agent_executor.invoke({"input": "What is the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': 'What is the paper 1605.08386 about?',
 'output': 'The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}